In [1]:
import numpy as np
import matplotlib.pyplot as plt

import plotly
import plotly.graph_objs as go
import plotly.express as px
from plotly.subplots import make_subplots

In [2]:
def calc(x, z1, z2):
    T1 = 1
    T2 = 3
    k = 1
    dt = 0.01
    y1 = z1
    k1 = dt*((k/T1*x) - (y1/T1))
    k2 = dt*((k/T1*x) - 1/T1*(y1+k1/2))
    k3 = dt*((k/T1*x) - 1/T1*(y1+k2/2))
    k4 = dt*((k/T1*x) - 1/T1*(y1+k3))
    z1 = z1 + 1/6*(k1+2*k2+2*k3+k4)
    y2=z2
    m1 = dt*(1/T2*y1 - 1/T2*y2)
    m2 = dt*(1/T2*y1 - 1/T2*(y2+m1/2))
    m3 = dt*(1/T2*y1 - 1/T2*(y2+m2/2))
    m4 = dt*(1/T2*y1 - 1/T2*(y2+m3))
    z2 = z2 + 1/6*(m1 + 2*m2 + 2*m3 + m4)
    return y2, z1, z2

In [3]:
#t = 0.1
g = 1
L = 15
y, t_, x_prev = 0, 0, 0
y2, z1, z2 = 0, 0, 0
dt = 0.01
krit, prev_krit = 0, 0

#массивы для графиков
Y = []#ось у
X = []#ось х
H = []#запаздывание
G = []#без запаздывания
Krit = []#модульный критерий
L_mas = []#тактовый шаг
q1_1 = []
q2_1 = []
q3_1 = []

q1_2 = []


#запаздывание
# i, j - цикл для моделирования запаздывания
# l - итератор для внешнего цикла
i, j, l, ns, tau, h = 0, 0, 0, 0, 0.3, 0.3
ns=tau/dt
# И. и Д. регуляторы
integ, dx = 0, 0
q1, q2, q3 = 2, 0.5, 1.5#эталонный
#q1, q2, q3 = 5, 4, 3
prev_q1, prev_q2, prev_q3 = 0, 0, 0
Q1 = []
Q2 = []
Q3 = []

xi_1, xi_2, xi_3 = 0, 0, 0
integ_xi1, integ_xi2, integ_xi3 = 0, 0, 0
Xi1 = []
Xi1_graph = []
Xi2 = []
Xi2_graph = []
Xi3 = []
Xi3_graph = []
prev_xi1, prev_xi2, prev_xi3 = 0, 0, 0
y2_xi1, z1_xi1, z2_xi1 = 0, 0, 0
y2_xi2, z1_xi2, z2_xi2 = 0, 0, 0
y2_xi3, z1_xi3, z2_xi3 = 0, 0, 0

dI1, dI2, dI3 = 0, 0, 0
Di1 = []
Di2 = []
Di3 = []

#попробуй dI1**2 + dI2**2 + dI3**2 > 0.001

In [4]:
epsilon = 1
while (epsilon > 0.001):
    i, j = 0, 0
    Krit.clear()
    X.clear()
    Y.clear
    H.clear()
    G.clear()
    Xi1.clear()
    Xi2.clear()
    Xi3.clear()
    t_ = 0
    y, x, x_prev = 0, 0, 0
    krit = 0
    integ, diff = 0, 0
    integ_xi1, integ_xi2, integ_xi3 = 0, 0, 0
    diff_xi1, diff_xi2, diff_xi3 = 0, 0, 0
    pid = 0
    du_1, du_2, du_3 = 0, 0, 0
    xi_1, xi_2, xi_3 = 0, 0, 0
    y2, z1, z2 = 0, 0, 0
    y2_xi1, z1_xi1, z2_xi1 = 0, 0, 0
    y2_xi2, z1_xi2, z2_xi2 = 0, 0, 0
    y2_xi3, z1_xi3, z2_xi3 = 0, 0, 0
    prev_xi1, prev_xi2, prev_xi3 = 0, 0, 0
    dI1, dI2, dI3 = 0, 0, 0
###############################################################################################
    while(t_ <= L):
        integ_xi1 += xi_1*dt
        integ_xi2 += xi_2*dt
        integ_xi3 += xi_3*dt
        diff_xi1 = (xi_1 - prev_xi1)/dt
        diff_xi2 = (xi_2 - prev_xi2)/dt
        diff_xi3 = (xi_3 - prev_xi3)/dt

        x = g - y
        integ += x*dt
        diff = (x - x_prev)/dt
        pid = q1*x + q2*integ + q3*diff

        du_1 = x - q1*xi_1 - q2*integ_xi1 - q3*diff_xi1
        du_2 = -q1*xi_2 + integ - q2*integ_xi2 - q3*diff_xi2
        du_3 = -q1*xi_3 - q2*integ_xi3 + diff - q3*diff_xi3

        y2, z1, z2 = calc(pid, z1, z2)
        y2_xi1, z1_xi1, z2_xi1 = calc(du_1, z1_xi1, z2_xi1)
        y2_xi2, z1_xi2, z2_xi2 = calc(du_2, z1_xi2, z2_xi2)
        y2_xi3, z1_xi3, z2_xi3 = calc(du_3, z1_xi3, z2_xi3)

        krit += abs(x)*dt

        prev_xi1 = xi_1
        prev_xi2 = xi_2
        prev_xi3 = xi_3

        if i > ns:
            if j >= ns:
                j = 0
            j += 1
            y = Y[j]
            Y[j] = y2
        #
            xi_1 = Xi1[j]
            Xi1[j] = y2_xi1
        #
            xi_2 = Xi2[j]
            Xi2[j] = y2_xi2
        #
            xi_3 = Xi3[j]
            Xi3[j] = y2_xi3
        else:
            Y.insert(i, y2)
            y = 0
        #
            Xi1.insert(i, y2_xi1)
            xi_1 = 0
        #
            Xi2.insert(i, y2_xi2)
            xi_2 = 0
        #
            Xi3.insert(i, y2_xi3)
            xi_3 = 0
        X.insert(i, t_)
        H.insert(i, y)
        G.insert(i, y2)
        Xi1.insert(i, xi_1)
        Xi2.insert(i, xi_2)
        Xi3.insert(i, xi_3)
        Krit.insert(i, krit)
        i += 1

        dI1 -= xi_1 * np.sign(x) * dt
        dI2 -= xi_2 * np.sign(x) * dt
        dI3 -= xi_3 * np.sign(x) * dt
        x_prev = x
        t_ += dt
##################################################################
    prev_q1 = q1
    prev_q2 = q2
    prev_q3 = q3
    
    if (krit <= prev_krit):#это до вычисления q
        h *= 1.2
    else:
        h *= 0.5
    prev_krit = krit

    q1 = prev_q1 - h*np.sign(dI1)
    q2 = prev_q2 - h*np.sign(dI2)   
    q3 = prev_q3 - h*np.sign(dI3)
    if (q1 < 0):
        q1 = prev_q1
    if (q2 < 0):
        q2 = prev_q2
    if (q3 < 0):
        q3 = prev_q3
    
    Di1.insert(l, dI1)
    Di2.insert(l, dI2)
    Di3.insert(l, dI3) 
    
    Q1.insert(l, q1)
    Q2.insert(l, q2)
    Q3.insert(l, q3)
    #Krit.insert(l, krit)
    epsilon = dI1**2 + dI2**2 + dI3**2
    print(epsilon)
    l += 1
    print(l)
    L_mas.append(l)

fig = go.Figure()
fig.add_trace(go.Scatter(x = X, y = H, name = 'с запаздыванием'))
fig.add_trace(go.Scatter(x = X, y = G, name = 'без запаздывания'))
fig.add_trace(go.Scatter(x = X, y = Krit, name = 'крит. (модул.)'))
fig.show()
########################
fig1 = go.Figure()
fig1.add_trace(go.Scatter(x = X, y = Xi1, name = 'xi1'))
fig1.add_trace(go.Scatter(x = X, y = Xi2, name = 'xi2'))
fig1.add_trace(go.Scatter(x = X, y = Xi3, name = 'xi3'))
fig1.show()
########################
fig2 = go.Figure()
fig2.add_trace(go.Scatter(x = L_mas, y = Q1, name = 'q1'))
fig2.add_trace(go.Scatter(x = L_mas, y = Q2, name = 'q2'))
fig2.add_trace(go.Scatter(x = L_mas, y = Q3, name = 'q3'))
fig2.show()
##################################
fig3 = go.Figure()
fig3.add_trace(go.Scatter(x = L_mas, y = Di1, name = 'dI1'))
fig3.add_trace(go.Scatter(x = L_mas, y = Di2, name = 'dI2'))
fig3.add_trace(go.Scatter(x = L_mas, y = Di3, name = 'dI3'))
fig3.show()

14.376984729874225
1
1.3562643835318051
2
14.038429993229435
3
8.515309134863198
4
0.19750367014982667
5
8.518833988834619
6
6.102151010502469
7
0.40792778878934266
8
0.2769152182329979
9
4.2499870619358004
10
0.46768742666535595
11
0.11477995489655843
12
0.2668777832902629
13
0.42585363334306825
14
0.11786473165322628
15
2.4346610095224372
16
0.622819963564159
17
0.06710898156871026
18
0.66316651279077
19
0.5477138637394985
20
0.5221449075645458
21
0.5411895021038107
22
1.5984499095509126
23
1.3524363109822646
24
0.10473366692774552
25
0.20235528278743992
26
0.719438501005999
27
0.14780572622270607
28
0.126738869509611
29
1.0252923625666661
30
0.20633269212131736
31
0.04950712787935824
32
0.40861962954813724
33
0.24718706073215363
34
0.41867327319267766
35
0.030394759742060917
36
0.40370859765586786
37
0.044912150887697816
38
0.48745205430299376
39
0.21397268683527026
40
0.05415897267491117
41
0.16868297512903824
42
0.2783149977014783
43
0.025564115759075752
44
0.3649835946012807
45
0

In [5]:
q1, q2, q3 = 5, 4, 3
prev_q1, prev_q2, prev_q3 = 0, 0, 0
x = 0 
l = 0
epsilon = 1
L_mas.clear()
Di1.clear()
Di2.clear()
Di3.clear()
Q1.clear()
Q2.clear()
Q3.clear()
while (epsilon > 0.001):
    i, j = 0, 0
    Krit.clear()
    X.clear()
    Y.clear
    H.clear()
    G.clear()
    Xi1.clear()
    Xi2.clear()
    Xi3.clear()
    t_ = 0
    y, x, x_prev = 0, 0, 0
    krit = 0
    integ, diff = 0, 0
    integ_xi1, integ_xi2, integ_xi3 = 0, 0, 0
    diff_xi1, diff_xi2, diff_xi3 = 0, 0, 0
    pid = 0
    du_1, du_2, du_3 = 0, 0, 0
    xi_1, xi_2, xi_3 = 0, 0, 0
    y2, z1, z2 = 0, 0, 0
    y2_xi1, z1_xi1, z2_xi1 = 0, 0, 0
    y2_xi2, z1_xi2, z2_xi2 = 0, 0, 0
    y2_xi3, z1_xi3, z2_xi3 = 0, 0, 0
    prev_xi1, prev_xi2, prev_xi3 = 0, 0, 0
    dI1, dI2, dI3 = 0, 0, 0
###############################################################################################
    while(t_ <= L):
        integ_xi1 += xi_1*dt
        integ_xi2 += xi_2*dt
        integ_xi3 += xi_3*dt
        diff_xi1 = (xi_1 - prev_xi1)/dt
        diff_xi2 = (xi_2 - prev_xi2)/dt
        diff_xi3 = (xi_3 - prev_xi3)/dt

        x = g - y
        integ += x*dt
        diff = (x - x_prev)/dt
        pid = q1*x + q2*integ + q3*diff

        du_1 = x - q1*xi_1 - q2*integ_xi1 - q3*diff_xi1
        du_2 = -q1*xi_2 + integ - q2*integ_xi2 - q3*diff_xi2
        du_3 = -q1*xi_3 - q2*integ_xi3 + diff - q3*diff_xi3

        y2, z1, z2 = calc(pid, z1, z2)
        y2_xi1, z1_xi1, z2_xi1 = calc(du_1, z1_xi1, z2_xi1)
        y2_xi2, z1_xi2, z2_xi2 = calc(du_2, z1_xi2, z2_xi2)
        y2_xi3, z1_xi3, z2_xi3 = calc(du_3, z1_xi3, z2_xi3)

        krit += abs(x)*dt

        prev_xi1 = xi_1
        prev_xi2 = xi_2
        prev_xi3 = xi_3

        if i > ns:
            if j >= ns:
                j = 0
            j += 1
            y = Y[j]
            Y[j] = y2
        #
            xi_1 = Xi1[j]
            Xi1[j] = y2_xi1
        #
            xi_2 = Xi2[j]
            Xi2[j] = y2_xi2
        #
            xi_3 = Xi3[j]
            Xi3[j] = y2_xi3
        else:
            Y.insert(i, y2)
            y = 0
        #
            Xi1.insert(i, y2_xi1)
            xi_1 = 0
        #
            Xi2.insert(i, y2_xi2)
            xi_2 = 0
        #
            Xi3.insert(i, y2_xi3)
            xi_3 = 0
        X.insert(i, t_)
        H.insert(i, y)
        G.insert(i, y2)
        Xi1.insert(i, xi_1)
        Xi2.insert(i, xi_2)
        Xi3.insert(i, xi_3)
        Krit.insert(i, krit)
        i += 1

        dI1 -= xi_1 * np.sign(x) * dt
        dI2 -= xi_2 * np.sign(x) * dt
        dI3 -= xi_3 * np.sign(x) * dt
        x_prev = x
        t_ += dt
##################################################################
    prev_q1 = q1
    prev_q2 = q2
    prev_q3 = q3
    
    if (krit <= prev_krit):#это до вычисления q
        h *= 1.2
    else:
        h *= 0.5
    prev_krit = krit

    q1 = prev_q1 - h*np.sign(dI1)
    q2 = prev_q2 - h*np.sign(dI2)   
    q3 = prev_q3 - h*np.sign(dI3)
    if (q1 < 0):
        q1 = prev_q1
    if (q2 < 0):
        q2 = prev_q2
    if (q3 < 0):
        q3 = prev_q3
    
    Di1.insert(l, dI1)
    Di2.insert(l, dI2)
    Di3.insert(l, dI3) 
    
    Q1.insert(l, q1)
    Q2.insert(l, q2)
    Q3.insert(l, q3)
    #Krit.insert(l, krit)
    epsilon = dI1**2 + dI2**2 + dI3**2
    print(epsilon)
    l += 1
    print(l)
    L_mas.append(l)

fig = go.Figure()
fig.add_trace(go.Scatter(x = X, y = H, name = 'с запаздыванием'))
fig.add_trace(go.Scatter(x = X, y = G, name = 'без запаздывания'))
fig.add_trace(go.Scatter(x = X, y = Krit, name = 'крит. (модул.)'))
fig.show()
########################
fig1 = go.Figure()
fig1.add_trace(go.Scatter(x = X, y = Xi1, name = 'xi1'))
fig1.add_trace(go.Scatter(x = X, y = Xi2, name = 'xi2'))
fig1.add_trace(go.Scatter(x = X, y = Xi3, name = 'xi3'))
fig1.show()
########################
fig2 = go.Figure()
fig2.add_trace(go.Scatter(x = L_mas, y = Q1, name = 'q1'))
fig2.add_trace(go.Scatter(x = L_mas, y = Q2, name = 'q2'))
fig2.add_trace(go.Scatter(x = L_mas, y = Q3, name = 'q3'))
fig2.show()
##################################
axis_dI1 = np.array(Di1)
axis_dI2 = np.array(Di2)
axis_dI3 = np.array(Di3)
fig3 = go.Figure()
fig3.add_trace(go.Scatter(x = L_mas, y = Di1, name = 'dI1'))
fig3.add_trace(go.Scatter(x = L_mas, y = Di2, name = 'dI2'))
fig3.add_trace(go.Scatter(x = L_mas, y = Di3, name = 'dI3'))
fig3.show()

0.6585167724394719
1
0.6421184178195181
2
0.6251584549737303
3
0.6068596465752463
4
0.5842676398170068
5
0.5581607711956476
6
0.5294142744908494
7
0.49582827319632755
8
0.46016605896593943
9
0.42027870391608824
10
0.3812520254983685
11
0.33685928130729015
12
0.2912593588248799
13
0.24556544853534956
14
0.20055560078127643
15
0.15924790513403794
16
0.12167302576572694
17
0.08952558542816254
18
0.06459399250746216
19
0.046747208984822715
20
0.04035893335874249
21
0.05695553181224886
22
0.06529559368758883
23
0.02796549585230451
24
0.023185048499459935
25
0.08677724047457465
26
0.023125080550074208
27
0.09144878753158146
28
0.07817055336945336
29
0.024219103277647945
30
0.08103128298445154
31
0.010762937410558965
32
0.026037625689606356
33
0.02405555515907416
34
0.06979076505517796
35
0.0002953974425464995
36


In [6]:
q1, q2, q3 = 1, 0.1, 0.8
prev_q1, prev_q2, prev_q3 = 0, 0, 0
x = 0 
l = 0
epsilon = 1
L_mas.clear()
Di1.clear()
Di2.clear()
Di3.clear()
Q1.clear()
Q2.clear()
Q3.clear()
while (epsilon > 0.001):
    i, j = 0, 0
    Krit.clear()
    X.clear()
    Y.clear
    H.clear()
    G.clear()
    Xi1.clear()
    Xi2.clear()
    Xi3.clear()
    t_ = 0
    y, x, x_prev = 0, 0, 0
    krit = 0
    integ, diff = 0, 0
    integ_xi1, integ_xi2, integ_xi3 = 0, 0, 0
    diff_xi1, diff_xi2, diff_xi3 = 0, 0, 0
    pid = 0
    du_1, du_2, du_3 = 0, 0, 0
    xi_1, xi_2, xi_3 = 0, 0, 0
    y2, z1, z2 = 0, 0, 0
    y2_xi1, z1_xi1, z2_xi1 = 0, 0, 0
    y2_xi2, z1_xi2, z2_xi2 = 0, 0, 0
    y2_xi3, z1_xi3, z2_xi3 = 0, 0, 0
    prev_xi1, prev_xi2, prev_xi3 = 0, 0, 0
    dI1, dI2, dI3 = 0, 0, 0
###############################################################################################
    while(t_ <= L):
        integ_xi1 += xi_1*dt
        integ_xi2 += xi_2*dt
        integ_xi3 += xi_3*dt
        diff_xi1 = (xi_1 - prev_xi1)/dt
        diff_xi2 = (xi_2 - prev_xi2)/dt
        diff_xi3 = (xi_3 - prev_xi3)/dt

        x = g - y
        integ += x*dt
        diff = (x - x_prev)/dt
        pid = q1*x + q2*integ + q3*diff

        du_1 = x - q1*xi_1 - q2*integ_xi1 - q3*diff_xi1
        du_2 = -q1*xi_2 + integ - q2*integ_xi2 - q3*diff_xi2
        du_3 = -q1*xi_3 - q2*integ_xi3 + diff - q3*diff_xi3

        y2, z1, z2 = calc(pid, z1, z2)
        y2_xi1, z1_xi1, z2_xi1 = calc(du_1, z1_xi1, z2_xi1)
        y2_xi2, z1_xi2, z2_xi2 = calc(du_2, z1_xi2, z2_xi2)
        y2_xi3, z1_xi3, z2_xi3 = calc(du_3, z1_xi3, z2_xi3)

        krit += abs(x)*dt

        prev_xi1 = xi_1
        prev_xi2 = xi_2
        prev_xi3 = xi_3

        if i > ns:
            if j >= ns:
                j = 0
            j += 1
            y = Y[j]
            Y[j] = y2
        #
            xi_1 = Xi1[j]
            Xi1[j] = y2_xi1
        #
            xi_2 = Xi2[j]
            Xi2[j] = y2_xi2
        #
            xi_3 = Xi3[j]
            Xi3[j] = y2_xi3
        else:
            Y.insert(i, y2)
            y = 0
        #
            Xi1.insert(i, y2_xi1)
            xi_1 = 0
        #
            Xi2.insert(i, y2_xi2)
            xi_2 = 0
        #
            Xi3.insert(i, y2_xi3)
            xi_3 = 0
        X.insert(i, t_)
        H.insert(i, y)
        G.insert(i, y2)
        Xi1.insert(i, xi_1)
        Xi2.insert(i, xi_2)
        Xi3.insert(i, xi_3)
        Krit.insert(i, krit)
        i += 1

        dI1 -= xi_1 * np.sign(x) * dt
        dI2 -= xi_2 * np.sign(x) * dt
        dI3 -= xi_3 * np.sign(x) * dt
        x_prev = x
        t_ += dt
##################################################################
    prev_q1 = q1
    prev_q2 = q2
    prev_q3 = q3
    
    if (krit <= prev_krit):#это до вычисления q
        h *= 1.2
    else:
        h *= 0.5
    prev_krit = krit

    q1 = prev_q1 - h*np.sign(dI1)
    q2 = prev_q2 - h*np.sign(dI2)   
    q3 = prev_q3 - h*np.sign(dI3)
    if (q1 < 0):
        q1 = prev_q1
    if (q2 < 0):
        q2 = prev_q2
    if (q3 < 0):
        q3 = prev_q3
    
    Di1.insert(l, dI1)
    Di2.insert(l, dI2)
    Di3.insert(l, dI3) 
    
    Q1.insert(l, q1)
    Q2.insert(l, q2)
    Q3.insert(l, q3)
    #Krit.insert(l, krit)
    epsilon = dI1**2 + dI2**2 + dI3**2
    print(epsilon)
    l += 1
    print(l)
    L_mas.append(l)

fig = go.Figure()
fig.add_trace(go.Scatter(x = X, y = H, name = 'с запаздыванием'))
fig.add_trace(go.Scatter(x = X, y = G, name = 'без запаздывания'))
fig.add_trace(go.Scatter(x = X, y = Krit, name = 'крит. (модул.)'))
fig.show()
########################
fig1 = go.Figure()
fig1.add_trace(go.Scatter(x = X, y = Xi1, name = 'xi1'))
fig1.add_trace(go.Scatter(x = X, y = Xi2, name = 'xi2'))
fig1.add_trace(go.Scatter(x = X, y = Xi3, name = 'xi3'))
fig1.show()
########################
fig2 = go.Figure()
fig2.add_trace(go.Scatter(x = L_mas, y = Q1, name = 'q1'))
fig2.add_trace(go.Scatter(x = L_mas, y = Q2, name = 'q2'))
fig2.add_trace(go.Scatter(x = L_mas, y = Q3, name = 'q3'))
fig2.show()
##################################
fig3 = go.Figure()
fig3.add_trace(go.Scatter(x = L_mas, y = Di1, name = 'dI1'))
fig3.add_trace(go.Scatter(x = L_mas, y = Di2, name = 'dI2'))
fig3.add_trace(go.Scatter(x = L_mas, y = Di3, name = 'dI3'))
fig3.show()

359.1169467318095
1
327.554005021164
2
293.2479890511305
3
257.90556232674675
4
220.61183635106568
5
182.35666522231568
6
144.47139276554458
7
108.56860978703743
8
76.38359542760652
9
7.957113787929901
10
3.438851319080474
11
47.974135745407324
12
35.51783296520834
13
2.3332668572536166
14
35.073702781641266
15
28.45456468108913
16
1.402058293815691
17
2.2079764185874464
18
22.3375432455031
19
2.572856298662966
20
0.6846301492655218
21
18.92360854201839
22
3.5178719686980386
23
0.5705267158283696
24
0.8143869844693759
25
3.226709358345489
26
0.44144985168863393
27
14.843272275185432
28
3.9464732630073844
29
0.5364439683086069
30
0.5436215454226778
31
3.4855357586449554
32
0.2775578311233507
33
11.197084564087481
34
3.7492174701013528
35
0.5089109180345744
36
0.3373903826738391
37
3.3043772637141244
38
0.8701397121966403
39
0.1962430006099365
40
0.39680446937913255
41
0.8437274230764592
42
0.1973113285873049
43
6.171545910666758
44
0.9906428904657015
45
0.1912287025883203
46
0.242893366